## High Performance Computation Using `Numba`

### The Weaknesses of Vectorization

- Highly memory-intensive when working with large amounts of data.  

- The set of algorithms that can be entirely vectorized is not universal.

### The Strength of  `Numba` 

- Specifically designed for numerical work and solves many of these problems.

- The key idea is to compile functions to native machine code instructions on the fly.  
 (Just in time compilation)

- When it succeeds, the compiled code is extremely fast.

- Can do other tricks such as automatic parallelization and multithreading.


### References

https://python-programming.quantecon.org/numba.html

https://numba.readthedocs.io/en/stable/index.html#

https://numba.readthedocs.io/en/stable/user/5minguide.html

In [1]:
import numpy as np
import quantecon as qe

### Compiling Functions

If the code is numerically orientated (does a lot of math), uses NumPy a lot and/or has a lot of loops, then Numba is often a good choice.

We apply one of the most fundamental of Numba’s JIT decorators, `@njit`, to speed up some functions.

**Example.** Consider simulating from the difference equation

$$
x_t = \mu + \rho x_{t-1} +\sigma \varepsilon_t,
\quad \{\varepsilon_t\} \overset{iid}{\sim} N(0,1).
$$

In [2]:
def ar1(x0=1., T=100, μ=1., ρ=0.7, σ=0.5, seed=None):
    x_vals = np.empty(T)
    x_vals[0] = x0
    
    if seed is not None:
        np.random.seed(seed)
        
    for t in range(T-1):
        x_vals[t+1] = μ + ρ*x_vals[t] + σ*np.random.randn()
        
    return x_vals

Next, we compile the funtion above using Numba decorator.

In [3]:
from numba import njit

In [4]:
@njit
def ar1_jitted(x0=1., T=100, μ=1., ρ=0.7, σ=0.5, seed=None):
    x_vals = np.empty(T)
    x_vals[0] = x0
    
    if seed is not None:
        np.random.seed(seed)
        
    for t in range(T-1):
        x_vals[t+1] = μ + ρ*x_vals[t] + σ*np.random.randn()
        
    return x_vals

In [5]:
with qe.Timer():
    ar1(T=1000_000)

0.56 seconds elapsed


In [7]:
with qe.Timer():
    ar1_jitted(T=1000_000)

0.01 seconds elapsed


We see that the Numba decorated function runs substantially faster.

### Compiling Classes

Numba is also quite effective at compiling classes.

If a class is successfully compiled, then its methods act as JIT-compiled functions.

#### Example: The Overlapping Generations Model

Recall that in the OLG model discussed previously, $k_t$ evolves according to the rule

$$
k_{t+1} = \frac{\beta}{1+\beta} (1-\alpha) k_t^\alpha
$$


In [8]:
from numba import float64
from numba.experimental import jitclass

In [9]:
olg_data = [
    ('α', float64),
    ('β', float64)
]

@jitclass(olg_data)
class OLGModel:
    
    def __init__(self, β=0.9, α=0.5):
        self.β, self.α = β, α
        
    def capital_supply(self, w):
        return self.β*w / (1 + self.β)
    
    def capital_demand(self, R):
        α = self.α
        return (α / R)**(1/(1 - α))
    
    def equilibrium(self, w):
        α, β = self.α, self.β
        ke = self.capital_supply(w)
        Re = α * ke**(α-1)
        return ke, Re
    
    def steady_state(self):
        α, β = self.α, self.β
        ks = (β*(1-α)/(1+β))**(1/(1-α))
        ws = (1+β) * ks / β
        Rs = α * (β*ws/(1+β))**(α-1)
        return ks, ws, Rs
    
    def k_update(self, k):
        α, β = self.α, self.β
        return β * (1-α) * (k**α) / (1+β)
    
    def simulate(self, T=100, k0=1):
        α, β = self.α, self.β
        k_sim = np.empty(T)
        k_sim[0] = k0
        for t in range(T-1):
            k_sim[t+1] = self.k_update(k_sim[t])
        w_sim = (1-α) * (k_sim**α)
        R_sim = α * (k_sim**(α-1))
        return k_sim, w_sim, R_sim

In [12]:
with qe.Timer():
    olg = OLGModel()
    olg.simulate(10_000_000)

0.49 seconds elapsed


### Supported Numpy Features

Numba can only compile a subset of Python. However, that subset is ever expanding. 

https://numba.readthedocs.io/en/stable/reference/numpysupported.html

In [13]:
def mean_func(X, axis=0):
    return np.mean(X, axis=axis)

In [14]:
X = np.random.randn(10,5)

In [15]:
mean_func(X)

array([ 0.62783614, -0.02332256,  0.25845897, -0.30874163, -0.04440263])

In [16]:
@njit
def mean_func(X, axis=0):
    return np.mean(X, axis=axis)

In [17]:
mean_func(X)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<function mean at 0x000002346B9A0FE0>) found for signature:
 
 >>> mean(array(float64, 2d, C), axis=Literal[int](0))
 
There are 2 candidate implementations:
[1m  - Of which 2 did not match due to:
  Overload in function 'array_mean': File: numba\np\arraymath.py: Line 425.
    With argument(s): '(array(float64, 2d, C), axis=int64)':[0m
[1m   Rejected as the implementation raised a specific error:
     TypingError: [1mgot an unexpected keyword argument 'axis'[0m[0m
  raised from D:\anaconda3\Lib\site-packages\numba\core\typing\templates.py:783
[0m
[0m[1mDuring: resolving callee type: Function(<function mean at 0x000002346B9A0FE0>)[0m
[0m[1mDuring: typing of call at C:\Users\Qingyin Ma\AppData\Local\Temp\ipykernel_13088\3041336866.py (3)
[0m
[1m
File "C:\Users\Qingyin Ma\AppData\Local\Temp\ipykernel_13088\3041336866.py", line 3:[0m
[1mdef mean_func(X,

### Automatic Parallelization

One can use Numba’s `prange` instead of `range` to specify that a loop can be parallelized. 

The user is required to make sure that the loop does not have cross iteration dependencies except for supported reductions.

https://numba.readthedocs.io/en/stable/user/parallel.html#numba-parallel


**Example.** Consider simulating from the difference equation again

$$
x_t = \mu + \rho x_{t-1} +\sigma \varepsilon_t,
\quad \{\varepsilon_t\} \overset{iid}{\sim} N(0,1)
$$

But this time simulating an $N\times T$ time series starting from $N$ different $x_0$ values.

In [18]:
from numba import prange

In [19]:
def ar1_vec(x0_vec, T=100):
    N = len(x0_vec)
    res = np.empty((N,T))
    
    for i in range(N):
        res[i,:] = ar1(x0_vec[i], T=T)
    
    return res

In [21]:
x0_vec = np.random.randn(100_000)

with qe.Timer():
    ar1_vec(x0_vec)

5.41 seconds elapsed


Automatic parallelization is realized by 
- setting the decorator as `@njit(parallel=True)` and
- using Numba's `prange` instead of Numpy's `range`.

In [22]:
@njit(parallel=True)
def ar1_vec_jitted(x0_vec, T=100):
    N = len(x0_vec)
    res = np.empty((N,T))
    
    for i in prange(N):
        res[i,:] = ar1_jitted(x0_vec[i], T=T)
    
    return res

In [25]:
with qe.Timer():
    ar1_vec_jitted(x0_vec)

0.02 seconds elapsed


The speed up of Numba's automatic parallelization is dramatic!